# Generics
Generics allow functions and classes that operate over arbitrary data types while preserving type safety.

For example, consider the identity function that returns the only argument. In the implementation below, the identity function is specified for strings and integers independently. 

In [1]:
def ident_int( x : Int) : Int =
    x
ident_int(1)
// ident_int("two")
def ident_string( x : String) : String =
    x
ident_string("two")

cmd1.sc:4: type mismatch;
 found   : String("two")
 required: Int
val res1_2 = ident_int("two")
                       ^Compilation Failed

: 

Generics allow more versitile and concise code.  The following identity function operates over any data type.

In [1]:
def ident[A]( x : A) : A = 
    x
ident(1)
ident("two")

defined function ident
res0_1: Int = 1
res0_2: String = "two"

Type safety is important with generics.  Before the program is run, a type checker can determine what type a generic (e.g., `A` in the previous function) is bound to.  For the call `ident(1)` the type checker can determine that the return type is an `Int` rather than a `String`.

The following identity function also operates on both `Int` and `String` but loses type information.  Note that `Any` is a type that includes all scala datatypes.

In [2]:
def ident_any( x : Any): Any =
    x
ident_any(1)
ident_any("two")

defined function ident_any
res1_1: Any = 1
res1_2: Any = "two"

With `ident_any`, we must check the type of the return value before we can do anything useful.  Note that the following code throws a type error.  The type checker cannot understand what `ident_any(1)` returns, even if it is evident to us.

In [2]:
1 + ident_any(1)

cmd2.sc:1: overloaded method value + with alternatives:
  (x: Double)Double <and>
  (x: Float)Float <and>
  (x: Long)Long <and>
  (x: Int)Int <and>
  (x: Char)Int <and>
  (x: Short)Int <and>
  (x: Byte)Int <and>
  (x: String)String
 cannot be applied to (Any)
val res2 = 1 + ident_any(1)
             ^Compilation Failed

: 

Pattern matching may be used to fix this error.  However, solving the type error with pattern matching forces us to handle an error condition that is not possible.

In [ ]:
1 + (ident_any(1) match{
    case i : Int => i
    case _ => throw new IllegalStateException("This can never be thrown.")
})

Using the generic function gives the type checker a hint about how the types of the function parameters relate to the return of the function.

The signature of the ident function says "There exists a type A such that my parameter is a type A and my return is a type A". 
```scala
def ident[A]( x : A) : A
```

The type checker can figure out on its own that `ident(1)` returns an `Int`.

In [3]:
1+ident(1)

res2: Int = 2

### Example: List

When we create a list, it can be a list of any type.

List of Integers:
```scala
List(1,2,3)
```

List of Strings:
```scala
List("Eve", "Bob" ,"Alice")
```

The list implementation you saw before today was over integers:

In [4]:
sealed trait ListInt
case object EmptyInt extends ListInt
case class ConsInt(v:Int, rest: ListInt) extends ListInt

ConsInt(1, ConsInt(2, ConsInt(3, EmptyInt)))

defined trait ListInt
defined object EmptyInt
defined class ConsInt
res3_3: ConsInt = ConsInt(1, ConsInt(2, ConsInt(3, EmptyInt)))

Implement `MyCons` using generics to complete a generic List implementation.

Note: we make `MyEmpty` a class here to simplify the generics.  Scala uses [lower type bounds](https://docs.scala-lang.org/tour/lower-type-bounds.html) allowing `Nil` to be a `case object` and allowing lists with different data types.  See the Scala implementation [here](https://github.com/scala/scala/blob/4c0f49c7de6ba48f2b0ae59e64ea94fabd82b4a7/src/library/scala/collection/immutable/List.scala#L594).

In [5]:
sealed trait MyList[A]
case class MyEmpty[A]() extends MyList[A]
??? // Your Solution Here

defined trait MyList
defined class MyEmpty
defined class MyCons

In [6]:
val list_int = MyCons( 1, MyCons(2, MyCons(3, MyEmpty())))
val list_double = MyCons( 1.1, MyCons(2.2, MyCons(3.3, MyEmpty())))

list_int: MyCons[Int] = MyCons(1, MyCons(2, MyCons(3, MyEmpty())))
list_double: MyCons[Double] = MyCons(1.1, MyCons(2.2, MyCons(3.3, MyEmpty())))

Note that our use of generics prevents us from writing a list with `Strings`s and `Int`s.

In [6]:
val bad_list = MyCons("1", MyCons(2.0, MyEmpty()))

cmd6.sc:1: type mismatch;
 found   : cmd6.this.cmd4.MyCons[Double]
 required: cmd6.this.cmd4.MyList[Any]
Note: Double <: Any, but trait MyList is invariant in type A.
You may wish to define A as +A instead. (SLS 4.5)
val bad_list = MyCons("1", MyCons(2.0, MyEmpty()))
                                 ^Compilation Failed

: 

The Scala `List` will allow different types.  We get a `List[Any]` due to the lower type bounds.

In [7]:
"1"::2.0::Nil

res6: List[Any] = List("1", 2.0)

### Example: map

Implement the map function over our list using generics.  For reference we have provided the int only map implementation.  Note that you may declare multiple generic types to use in a function with `def fun[A,B]`.  Think carefully about why map may need two generic types.

```scala
val strList: List[String] = List(1,2,3).map(_.toString)
```

In [ ]:
def map_int(l : MyList[Int], f: Int => Int) : MyList[Int] = 
    l match {
        case MyEmpty() => MyEmpty()
        case MyCons(h,t) => MyCons( f(h), map(t,f))
    }

In [8]:
??? // Your Solution Here

defined function map

In [9]:
assert(map( list_int, (x : Int) => "1" + x) == MyCons( "11", MyCons("12", MyCons("13", MyEmpty()))))
map[Int,Double](list_int, x => x)

res8_1: MyList[Double] = MyCons(1.0, MyCons(2.0, MyCons(3.0, MyEmpty())))

### Example: filter

Impliment a filter function over MyList.

In [11]:
??? // Your Solution Here

defined function filter

In [12]:
assert(filter( list_double, (x : Double)=> x > 2) == MyCons(2.2, MyCons(3.3, MyEmpty())))

### Example: fold

Impliment a fold function over MyList.

In [13]:
??? // Your Solution Here

defined function fold

In [14]:
val c = MyCons(2, MyCons(3, MyEmpty()))
assert(fold(0, c, (acc: Int, v: Int) => acc + v) == 5)

c: MyCons[Int] = MyCons(2, MyCons(3, MyEmpty()))